In [1]:
%pip install langchain

You should consider upgrading via the '/Users/giumast/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool, tool
from typing import List

import tensorflow as tf



In [5]:
class WineParams(BaseModel):
    query: List[List] = Field(default=[], description="A list of chemicals in our wine")
    
@tool("wine-quality-tool", return_direct=True)
def predict_wine_quality(query: list) -> int:
    """A tool that predicts the wine quality"""
    model = tf.keras.saving.load_model("model/model.h5")
    preds = model.predict(query)
    return preds[0][0]

print(f"Tool:         [{predict_wine_quality.name}]\nDescrizione:  {predict_wine_quality.description}\nInput schema: {predict_wine_quality.args}")



Tool:         [wine-quality-tool]
Descrizione:  wine-quality-tool(query: list) -> int - A tool that predicts the wine quality
Input schema: {'query': {'title': 'Query', 'type': 'array', 'items': {}}}


In [6]:
# alternative method

def xyz(query):
    return "pippo"

xyz_tool = StructuredTool.from_function(
    func=xyz,
    name="xyz-tool",
    description="Lorem Ipsum dolor sit amet.",
    # coroutine= ... eventuale metodo per chiamate asincrone, se disponibile
)

wine_tool = StructuredTool.from_function(
    func=predict_wine_quality,
    name="wine-tool",
    description="Use this tool to predict wine quality",
)

NameError: name 'Type' is not defined

In [ ]:
predict_wine_quality.run({"query": [[
        0.30973451, 
        0.11643836, 
        0.36      , 
        0.0890411 , 
        0.06010017,
        0.50746269, 
        0.16607774, 
        0.37547601, 
        0.41732283, 
        0.40718563,
        0.61538462
        ]]
    }
)

In [ ]:
%pip install numexpr

In [ ]:
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI  # pip install langchain-openai

model = ChatOpenAI(
    openai_api_key="", 
    temperature=.75, 
    max_tokens=1024, 
    request_timeout=30
)

tools = load_tools(["llm-math"], llm=model)  
tools.append(predict_wine_quality)
# tools = load_tools([predict_wine_quality], allow_dangerous_tools=True)


In [ ]:
from langchain.prompts import ChatPromptTemplate

system_prompt = """
You are an agent designed to give information about wines. You have access to a machine learning model, which you can use to answer to the user prompts. 
You might know the answer without using the model, but you should still use the model when possible to get the answer. 
If it does not seem like you cannot use the model to answer the question, just return "I don't know" as the answer. 
You have access to the following tools:\n\n{tools}\n\n
The way you use the tools is by getting a list of numbers from the user.\n
ALWAYS use the following format:\n\nQuestion: the input question you must answer\n
Thought: you should always think about what to do\n
Action:\n```\n$JSON_BLOB\n```\n
Observation: the result of the action\n
... (this Thought/Action/Observation can repeat N times)\n
Thought: I now know the final answer\n
Final Answer: the final answer to the original input question\n\n
Begin! Reminder to always use the exact characters `Final Answer` when responding.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])

In [ ]:
from langchain.agents.format_scratchpad import format_log_to_str  
from langchain_core.output_parsers import StrOutputParser

chat_model_with_stop = model.bind(stop=["\nFinal Answer"]) 

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "tools": lambda x: x["tool"]
    }
    | prompt
    | chat_model_with_stop
    | StrOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors=True
)

In [ ]:
agent_executor.invoke({"input": "Ciao"})

In [7]:
tf.__version__

'2.13.0'